In [1]:
import gradio as gr
import pandas as pd
from douzero.env.game import GameEnv
from douzero.evaluation.deep_agent import DeepAgent

# 常量定义
EnvCard2RealCard = {3: '3', 4: '4', 5: '5', 6: '6', 7: '7',
                    8: '8', 9: '9', 10: 'T', 11: 'J', 12: 'Q',
                    13: 'K', 14: 'A', 17: '2', 20: 'X', 30: 'D'}

RealCard2EnvCard = {'3': 3, '4': 4, '5': 5, '6': 6, '7': 7,
                    '8': 8, '9': 9, 'T': 10, 'J': 11, 'Q': 12,
                    'K': 13, 'A': 14, '2': 17, 'X': 20, 'D': 30}

AllEnvCard = [3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7,
              8, 8, 8, 8, 9, 9, 9, 9, 10, 10, 10, 10, 11, 11, 11, 11, 12,
              12, 12, 12, 13, 13, 13, 13, 14, 14, 14, 14, 17, 17, 17, 17, 20, 30]

allowed_cards = ["A", "2", "3", "4", "5", "6", "7", "8", "9", "T", "J", "Q", "K", "X", "D"]

custom_css = """
.gradio-container {
    max-width: 800px;
    margin: 0 auto !important;
    padding: 20px !important;
}
#title {
    text-align: center;
    font-size: 24px;
    margin-bottom: 30px !important;
}
#subtitle {
    text-align: left;
    font-size: 16px;
    margin-bottom: 2px !important;
}
"""

class DouDiZhuGame:
    def __init__(self):
        self.card_play_model_path_dict = {
            'landlord': "baselines/douzero_WP/landlord.ckpt",
            'landlord_up': "baselines/douzero_WP/landlord_up.ckpt",
            'landlord_down': "baselines/douzero_WP/landlord_down.ckpt"
        }
        self.env = None
        self.user_position = None
        self.play_order = None
        self.consecutive_passes = 0
        self.first_round = True
        self.last_played_position = None
        self.must_play = False
        self.recording = []
        self.current_hand_cards = ""
        self.current_suggestion = ""
        self.game_over = False
        self.winner = None

    def initialize_game(self, user_hand_cards_real, three_landlord_cards_real, user_position_code):
        """初始化游戏"""
        self.recording = []
        self.user_position = ['landlord_up', 'landlord', 'landlord_down'][user_position_code]
        
        # 转换手牌格式
        self.user_hand_cards_env = [RealCard2EnvCard[c] for c in list(user_hand_cards_real)]
        self.three_landlord_cards_env = [RealCard2EnvCard[c] for c in list(three_landlord_cards_real)]
        
        # 如果玩家是地主，将底牌加入手牌
        if self.user_position == "landlord":
            self.user_hand_cards_env.extend(self.three_landlord_cards_env)
            user_hand_cards_real += three_landlord_cards_real
        
        # 计算其他玩家的手牌
        self.other_hand_cards = []
        for i in set(AllEnvCard):
            self.other_hand_cards.extend([i] * (AllEnvCard.count(i) - self.user_hand_cards_env.count(i)))

        # 分配手牌
        if self.user_position == "landlord":
            landlord_up_cards = self.other_hand_cards[:17]
            landlord_down_cards = self.other_hand_cards[17:]
        elif self.user_position == "landlord_up":
            landlord_cards = self.other_hand_cards[:17] + self.three_landlord_cards_env
            landlord_down_cards = self.other_hand_cards[17:34]
        else:  # landlord_down
            landlord_cards = self.other_hand_cards[:17] + self.three_landlord_cards_env
            landlord_up_cards = self.other_hand_cards[17:34]

        self.card_play_data_list = {
            'three_landlord_cards': self.three_landlord_cards_env,
            'landlord_up': landlord_up_cards if self.user_position != "landlord_up" else self.user_hand_cards_env,
            'landlord': landlord_cards if self.user_position != "landlord" else self.user_hand_cards_env,
            'landlord_down': landlord_down_cards if self.user_position != "landlord_down" else self.user_hand_cards_env
        }

        # 设置出牌顺序
        self.play_order = 0 if self.user_position == "landlord" else 1 if self.user_position == "landlord_up" else 2

        # 创建游戏环境
        ai_players = [0, 0]
        ai_players[0] = self.user_position
        ai_players[1] = DeepAgent(self.user_position, self.card_play_model_path_dict[self.user_position])
        
        self.env = GameEnv(ai_players)
        self.env.card_play_init(self.card_play_data_list)
        
        # 重置游戏状态
        self.consecutive_passes = 0
        self.first_round = True
        self.last_played_position = None
        self.must_play = False
        self.game_over = False
        self.winner = None
        
        # 记录初始状态
        self.recording.append("游戏开始！")
        self.update_current_hand_cards()
        return "游戏初始化成功！"

    def update_current_hand_cards(self):
        """更新当前手牌显示"""
        if self.env and hasattr(self.env, 'info_sets'):
            self.current_hand_cards = ''.join([EnvCard2RealCard[c] for c in sorted(
                self.env.info_sets[self.user_position].player_hand_cards)])
        else:
            self.current_hand_cards = ""

    def get_ai_suggestion(self):
        """获取AI建议"""
        if not self.env or self.play_order != 0:
            return ""
            
        action_message = self.env.step(self.user_position)
        action_str = action_message["action"] if action_message["action"] else "不出"
        win_rate = action_message['win_rate']
        
        self.current_suggestion = f"{action_str} (胜率: {win_rate})"
        return self.current_suggestion

    def play_move(self, move_input):
        """处理出牌"""
        if self.game_over:
            return "游戏已结束，请重新开始新游戏"
            
        if not move_input:  # 不出
            if self.must_play:
                return "错误: 必须出牌，不能选择'不出'！"
            
            self.consecutive_passes += 1
            self.recording.append(f"[玩家出牌]: 不出")
            
            # 如果两个玩家都pass了，回到最后出牌的玩家
            if self.consecutive_passes >= 2 and self.last_played_position is not None:
                self.play_order = {'landlord':0, 'landlord_up':1, 'landlord_down':2}[self.last_played_position]
                self.consecutive_passes = 0
                self.must_play = True
                self.recording.append(f"[新回合] 由{self.last_played_position}开始新一轮出牌 (必须出牌)")
                return "\n".join(self.recording)
        else:
            # 验证牌型
            if not self.validate_card_pattern(move_input):
                return "错误: 牌型不符合规则！"
                
            # 转换牌型
            played_cards_env = [RealCard2EnvCard[c] for c in list(move_input)]
            self.env.step(self.user_position, played_cards_env)
            
            self.consecutive_passes = 0
            self.last_played_position = self.user_position
            self.must_play = False
            self.recording.append(f"[玩家出牌]: {move_input}")
        
        self.first_round = False
        self.play_order = (self.play_order + 1) % 3
        self.update_current_hand_cards()
        
        # 检查游戏是否结束
        if self.env.game_over:
            self.game_over = True
            self.winner = '农民' if self.env.winner == 'farmer' else '地主'
            self.recording.append(f"{self.winner}胜，本局结束!")
        
        return "\n".join(self.recording)

    def validate_card_pattern(self, cards_str):
        """验证牌型是否符合规则"""
        if not cards_str:
            return True

        cards = list(cards_str)
        card_count = {}
        for card in cards:
            card_count[card] = card_count.get(card, 0) + 1

        if len(cards) == 1:
            return True
        if len(cards) == 2 and len(card_count) == 1:
            return True
        if len(cards) == 3 and len(card_count) == 1:
            return True
        if len(cards) == 4 and (list(card_count.values()).count(3) == 1 and list(card_count.values()).count(1) == 1):
            return True
        if len(cards) == 4 and len(card_count) == 1:
            return True
        if len(cards) >= 5 and len(card_count) == len(cards) and self.is_sequence(cards):
            return True
        if len(cards) >= 6 and len(cards) % 2 == 0 and all(
                count == 2 for count in card_count.values()) and self.is_sequence(sorted(card_count.keys())):
            return True

        return False

    def is_sequence(self, cards):
        """检查是否是连续的牌"""
        order = ['3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A', '2']
        try:
            card_indices = [order.index(c) for c in cards]
        except ValueError:
            return False
        return all(card_indices[i] + 1 == card_indices[i + 1] for i in range(len(card_indices) - 1))

# 创建游戏实例
game = DouDiZhuGame()

def initialize_game(iniCard, bottomCard, place):
    """初始化游戏处理函数"""
    # 验证输入
    if len(iniCard) != 17:
        raise gr.Error("错误: 初始手牌必须是17张牌！")
    if len(bottomCard) != 3:
        raise gr.Error("错误: 底牌必须是3张牌！")
    for card in list(iniCard):
        if card not in allowed_cards:
            raise gr.Error(f"错误: 无效的牌 '{card}'")
    for card in list(bottomCard):
        if card not in allowed_cards:
            raise gr.Error(f"错误: 无效的牌 '{card}'")
    
    # 转换位置选项
    place_code = 0 if place == "Upper Neighbour" else 1 if place == "Landlord" else 2
    
    # 初始化游戏
    result = game.initialize_game(iniCard.upper(), bottomCard.upper(), place_code)
    
    # 获取初始手牌和AI建议
    hand_cards = game.current_hand_cards
    ai_suggestion = game.get_ai_suggestion()
    
    return result, hand_cards, ai_suggestion, gr.Textbox(interactive=True)

def process_move(move_input):
    """处理出牌函数"""
    if not game.env:
        return "错误: 游戏未初始化，请先提交初始手牌", "", ""
    
    move_input = move_input.upper()
    for card in list(move_input):
        if card not in allowed_cards:
            return "错误: 牌型包含无效的牌", "", ""
    
    # 处理出牌
    recording = game.play_move(move_input)
    hand_cards = game.current_hand_cards
    
    # 如果是玩家回合，获取AI建议
    ai_suggestion = ""
    if game.play_order == 0 and not game.game_over:
        ai_suggestion = game.get_ai_suggestion()
    
    return recording, hand_cards, ai_suggestion

with gr.Blocks(css=custom_css) as demo: 
    # 标题
    gr.Markdown("""<h1 id="title">斗地主AI辅助工具</h1>""")
    
    # 初始化部分
    gr.Markdown("""<h3 id="subtitle">初始设置</h3>""")
    with gr.Row():
        iniCard = gr.Textbox(label="你的手牌 (17张)", max_length=17)
        bottomCard = gr.Textbox(label="底牌 (3张)", max_length=3)
    place = gr.Radio(choices=["Upper Neighbour (地主上家)", "Landlord (地主)", "Lower Neighbour (地主下家)"], 
                    label="你的位置", value="Landlord (地主)")
    submit_btn = gr.Button("开始游戏", variant="primary")
    
    # 游戏记录部分
    gr.Markdown("""<h3 id="subtitle">游戏记录</h3>""")
    recording = gr.Textbox(label="游戏记录", interactive=False, lines=10)
    
    # 游戏操作部分
    gr.Markdown("""<h3 id="subtitle">游戏操作</h3>""")
    with gr.Row():
        hand_cards = gr.Textbox(label="当前手牌", interactive=False)
        ai_suggestion = gr.Textbox(label="AI建议", interactive=False)
    with gr.Row():
        move_input = gr.Textbox(label="你的出牌 (留空表示'不出')", interactive=False)
        move_btn = gr.Button("出牌", variant="primary")
    
    # 结果部分
    gr.Markdown("""<h3 id="subtitle">游戏结果</h3>""")
    result = gr.Textbox(label="结果", interactive=False)

    # 事件处理
    submit_btn.click(
        fn=initialize_game,
        inputs=[iniCard, bottomCard, place],
        outputs=[recording, hand_cards, ai_suggestion, move_input]
    )
    
    move_btn.click(
        fn=process_move,
        inputs=[move_input],
        outputs=[recording, hand_cards, ai_suggestion]
    )

demo.launch(inbrowser=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


C:\Users\Administrator\Desktop\Homework\AI\homework\ddz\DouZero_For_HappyDouDiZhu-master\douzero\evaluation\deep_agent.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  p

In [5]:
import gradio as gr
from douzero.env.game import GameEnv
from douzero.evaluation.deep_agent import DeepAgent

# 常量定义
EnvCard2RealCard = {3: '3', 4: '4', 5: '5', 6: '6', 7: '7',
                    8: '8', 9: '9', 10: 'T', 11: 'J', 12: 'Q',
                    13: 'K', 14: 'A', 17: '2', 20: 'X', 30: 'D'}

RealCard2EnvCard = {'3': 3, '4': 4, '5': 5, '6': 6, '7': 7,
                    '8': 8, '9': 9, 'T': 10, 'J': 11, 'Q': 12,
                    'K': 13, 'A': 14, '2': 17, 'X': 20, 'D': 30}

AllEnvCard = [3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7,
              8, 8, 8, 8, 9, 9, 9, 9, 10, 10, 10, 10, 11, 11, 11, 11, 12,
              12, 12, 12, 13, 13, 13, 13, 14, 14, 14, 14, 17, 17, 17, 17, 20, 30]

allowed_cards = ["A", "2", "3", "4", "5", "6", "7", "8", "9", "T", "J", "Q", "K", "X", "D"]

custom_css = """
.gradio-container {
    max-width: 900px;
    margin: 0 auto !important;
    padding: 20px !important;
}
#title {
    text-align: center;
    font-size: 24px;
    margin-bottom: 30px !important;
}
#subtitle {
    text-align: left;
    font-size: 16px;
    margin-bottom: 2px !important;
}
.player-area {
    border: 1px solid #ccc;
    border-radius: 5px;
    padding: 10px;
    margin-bottom: 10px;
}
.landlord {
    background-color: #ffebee;
}
.farmer {
    background-color: #e8f5e9;
}
"""

class DouDiZhuGame:
    def __init__(self):
        self.card_play_model_path_dict = {
            'landlord': "baselines/douzero_WP/landlord.ckpt",
            'landlord_up': "baselines/douzero_WP/landlord_up.ckpt",
            'landlord_down': "baselines/douzero_WP/landlord_down.ckpt"
        }
        self.env = None
        self.user_position = None
        self.play_order = None
        self.consecutive_passes = 0
        self.first_round = True
        self.last_played_position = None
        self.must_play = False
        self.recording = []
        self.current_hand_cards = ""
        self.current_suggestion = ""
        self.game_over = False
        self.winner = None

    def initialize_game(self, user_hand_cards_real, three_landlord_cards_real, user_position_code):
        """初始化游戏"""
        self.recording = []
        self.user_position = ['landlord_up', 'landlord', 'landlord_down'][user_position_code]
        
        # 转换手牌格式
        self.user_hand_cards_env = [RealCard2EnvCard[c] for c in list(user_hand_cards_real)]
        self.three_landlord_cards_env = [RealCard2EnvCard[c] for c in list(three_landlord_cards_real)]
        
        # 如果玩家是地主，将底牌加入手牌
        if self.user_position == "landlord":
            self.user_hand_cards_env.extend(self.three_landlord_cards_env)
            user_hand_cards_real += three_landlord_cards_real
        
        # 计算其他玩家的手牌
        self.other_hand_cards = []
        for i in set(AllEnvCard):
            self.other_hand_cards.extend([i] * (AllEnvCard.count(i) - self.user_hand_cards_env.count(i)))

        # 分配手牌
        if self.user_position == "landlord":
            landlord_up_cards = self.other_hand_cards[:17]
            landlord_down_cards = self.other_hand_cards[17:]
        elif self.user_position == "landlord_up":
            landlord_cards = self.other_hand_cards[:17] + self.three_landlord_cards_env
            landlord_down_cards = self.other_hand_cards[17:34]
        else:  # landlord_down
            landlord_cards = self.other_hand_cards[:17] + self.three_landlord_cards_env
            landlord_up_cards = self.other_hand_cards[17:34]

        self.card_play_data_list = {
            'three_landlord_cards': self.three_landlord_cards_env,
            'landlord_up': landlord_up_cards if self.user_position != "landlord_up" else self.user_hand_cards_env,
            'landlord': landlord_cards if self.user_position != "landlord" else self.user_hand_cards_env,
            'landlord_down': landlord_down_cards if self.user_position != "landlord_down" else self.user_hand_cards_env
        }

        # 设置出牌顺序
        self.play_order = 0 if self.user_position == "landlord" else 1 if self.user_position == "landlord_up" else 2

        # 创建游戏环境
        ai_players = [0, 0]
        ai_players[0] = self.user_position
        ai_players[1] = DeepAgent(self.user_position, self.card_play_model_path_dict[self.user_position])
        
        self.env = GameEnv(ai_players)
        self.env.card_play_init(self.card_play_data_list)
        
        # 重置游戏状态
        self.consecutive_passes = 0
        self.first_round = True
        self.last_played_position = None
        self.must_play = False
        self.game_over = False
        self.winner = None
        
        # 记录初始状态
        position_name = "地主" if self.user_position == "landlord" else "农民(上家)" if self.user_position == "landlord_up" else "农民(下家)"
        self.recording.append(f"游戏开始！你的位置: {position_name}")
        self.update_current_hand_cards()
        return "游戏初始化成功！"

    def update_current_hand_cards(self):
        """更新当前手牌显示"""
        if self.env and hasattr(self.env, 'info_sets'):
            self.current_hand_cards = ''.join([EnvCard2RealCard[c] for c in sorted(
                self.env.info_sets[self.user_position].player_hand_cards)])
        else:
            self.current_hand_cards = ""

    def get_ai_suggestion(self):
        """获取AI建议"""
        if not self.env or self.play_order != 0:
            return ""
            
        action_message = self.env.step(self.user_position)
        action_str = action_message["action"] if action_message["action"] else "不出"
        win_rate = action_message['win_rate']
        
        self.current_suggestion = f"{action_str} (胜率: {win_rate})"
        return self.current_suggestion

    def play_move(self, move_input, player_type):
        """处理出牌"""
        if self.game_over:
            return "游戏已结束，请重新开始新游戏"
            
        # 确定当前应该出牌的玩家
        current_player = ""
        if player_type == "player":
            current_player = self.user_position
        elif player_type == "up":
            current_player = "landlord_up" if self.user_position != "landlord_up" else None
        elif player_type == "down":
            current_player = "landlord_down" if self.user_position != "landlord_down" else None
            
        if not move_input:  # 不出
            if self.must_play:
                return "错误: 必须出牌，不能选择'不出'！"
            
            self.consecutive_passes += 1
            player_name = "地主" if current_player == "landlord" else "农民(上家)" if current_player == "landlord_up" else "农民(下家)"
            self.recording.append(f"[{player_name}出牌]: 不出")
            
            # 如果两个玩家都pass了，回到最后出牌的玩家
            if self.consecutive_passes >= 2 and self.last_played_position is not None:
                self.play_order = {'landlord':0, 'landlord_up':1, 'landlord_down':2}[self.last_played_position]
                self.consecutive_passes = 0
                self.must_play = True
                last_player_name = "地主" if self.last_played_position == "landlord" else "农民(上家)" if self.last_played_position == "landlord_up" else "农民(下家)"
                self.recording.append(f"[新回合] 由{last_player_name}开始新一轮出牌 (必须出牌)")
                return "\n".join(self.recording)
        else:
            # 验证牌型
            if not self.validate_card_pattern(move_input):
                return "错误: 牌型不符合规则！"
                
            # 转换牌型
            played_cards_env = [RealCard2EnvCard[c] for c in list(move_input)]
            self.env.step(current_player, played_cards_env)
            
            self.consecutive_passes = 0
            self.last_played_position = current_player
            self.must_play = False
            player_name = "地主" if current_player == "landlord" else "农民(上家)" if current_player == "landlord_up" else "农民(下家)"
            self.recording.append(f"[{player_name}出牌]: {move_input}")
        
        self.first_round = False
        self.play_order = (self.play_order + 1) % 3
        self.update_current_hand_cards()
        
        # 检查游戏是否结束
        if self.env.game_over:
            self.game_over = True
            self.winner = '农民' if self.env.winner == 'farmer' else '地主'
            self.recording.append(f"{self.winner}胜，本局结束!")
        
        return "\n".join(self.recording)

    def validate_card_pattern(self, cards_str):
        """验证牌型是否符合规则"""
        if not cards_str:
            return True

        cards = list(cards_str)
        card_count = {}
        for card in cards:
            card_count[card] = card_count.get(card, 0) + 1
        if len(cards) == 2 and set(cards) == {'X', 'D'}:
            return True
        if len(cards) == 1:
            return True
        if len(cards) == 2 and len(card_count) == 1:
            return True
        if len(cards) == 3 and len(card_count) == 1:
            return True
        if len(cards) == 4 and (list(card_count.values()).count(3) == 1 and list(card_count.values()).count(1) == 1):
            return True
        if len(cards) == 4 and len(card_count) == 1:
            return True
        if len(cards) >= 5 and len(card_count) == len(cards) and self.is_sequence(cards):
            return True
        if len(cards) >= 6 and len(cards) % 2 == 0 and all(
                count == 2 for count in card_count.values()) and self.is_sequence(sorted(card_count.keys())):
            return True

        return False

    def is_sequence(self, cards):
        """检查是否是连续的牌"""
        order = ['3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A', '2']
        try:
            card_indices = [order.index(c) for c in cards]
        except ValueError:
            return False
        return all(card_indices[i] + 1 == card_indices[i + 1] for i in range(len(card_indices) - 1))

# 创建游戏实例
game = DouDiZhuGame()

def initialize_game(iniCard, bottomCard, place):
    """初始化游戏处理函数"""
    # 验证输入
    if len(iniCard) != 17:
        raise gr.Error("错误: 初始手牌必须是17张牌！")
    if len(bottomCard) != 3:
        raise gr.Error("错误: 底牌必须是3张牌！")
    for card in list(iniCard):
        if card not in allowed_cards:
            raise gr.Error(f"错误: 无效的牌 '{card}'")
    for card in list(bottomCard):
        if card not in allowed_cards:
            raise gr.Error(f"错误: 无效的牌 '{card}'")
    
    # 转换位置选项
    place_code = 0 if place == "Upper Neighbour (地主上家)" else 1 if place == "Landlord (地主)" else 2
    
    # 初始化游戏
    result = game.initialize_game(iniCard.upper(), bottomCard.upper(), place_code)
    
    # 获取初始手牌和AI建议
    hand_cards = game.current_hand_cards
    ai_suggestion = game.get_ai_suggestion()
    
    # 根据玩家位置确定哪些输入框应该激活
    player_active = game.play_order == 0
    up_active = game.play_order == 1
    down_active = game.play_order == 2
    
    return (
        result, 
        hand_cards, 
        ai_suggestion,
        gr.Textbox(interactive=player_active),
        gr.Textbox(interactive=up_active),
        gr.Textbox(interactive=down_active),
        gr.Button(interactive=player_active, variant="primary" if player_active else "secondary"),
        gr.Button(interactive=up_active, variant="primary" if up_active else "secondary"),
        gr.Button(interactive=down_active, variant="primary" if down_active else "secondary")
    )

def process_move(move_input, player_type):
    """处理出牌函数"""
    if not game.env:
        return "错误: 游戏未初始化，请先提交初始手牌", "", "", "", "", "", "", "", ""
    
    move_input = move_input.upper() if move_input else ""
    for card in list(move_input):
        if card not in allowed_cards and move_input:
            return "错误: 牌型包含无效的牌", "", "", "", "", "", "", "", ""
    
    # 处理出牌
    recording = game.play_move(move_input, player_type)
    hand_cards = game.current_hand_cards
    
    # 如果是玩家回合，获取AI建议
    ai_suggestion = ""
    if game.play_order == 0 and not game.game_over:
        ai_suggestion = game.get_ai_suggestion()
    
    # 根据当前出牌顺序确定哪些输入框应该激活
    player_active = game.play_order == 0 and not game.game_over
    up_active = game.play_order == 1 and not game.game_over
    down_active = game.play_order == 2 and not game.game_over
    
    return (
        recording, 
        hand_cards, 
        ai_suggestion,
        gr.Textbox(interactive=player_active, value=""),
        gr.Textbox(interactive=up_active, value=""),
        gr.Textbox(interactive=down_active, value=""),
        gr.Button(interactive=player_active, variant="primary" if player_active else "secondary"),
        gr.Button(interactive=up_active, variant="primary" if up_active else "secondary"),
        gr.Button(interactive=down_active, variant="primary" if down_active else "secondary")
    )

with gr.Blocks(css=custom_css) as demo: 
    # 标题
    gr.Markdown("""<h1 id="title">斗地主AI辅助工具</h1>""")
    
    # 初始化部分
    gr.Markdown("""<h3 id="subtitle">初始设置</h3>""")
    with gr.Row():
        iniCard = gr.Textbox(label="你的手牌 (17张)", max_length=17)
        bottomCard = gr.Textbox(label="底牌 (3张)", max_length=3)
    place = gr.Radio(
        choices=["Upper Neighbour (地主上家)", "Landlord (地主)", "Lower Neighbour (地主下家)"], 
        label="你的位置", 
        value="Landlord (地主)"
    )
    submit_btn = gr.Button("开始游戏", variant="primary")
    
    # 游戏记录部分
    gr.Markdown("""<h3 id="subtitle">游戏记录</h3>""")
    recording = gr.Textbox(label="游戏记录", interactive=False, lines=10)
    
    # 游戏操作部分
    gr.Markdown("""<h3 id="subtitle">游戏操作</h3>""")
    with gr.Row():
        hand_cards = gr.Textbox(label="当前手牌", interactive=False)
        ai_suggestion = gr.Textbox(label="AI建议", interactive=False)
    
    # 玩家出牌区域
    with gr.Column() as player_area:
        gr.Markdown("### 玩家出牌")
        with gr.Row():
            player_input = gr.Textbox(label="你的出牌 (留空表示'不出')", interactive=False)
            player_btn = gr.Button("出牌", interactive=False)
    
    # 上家出牌区域
    with gr.Column() as up_area:
        gr.Markdown("### 上家出牌")
        with gr.Row():
            up_input = gr.Textbox(label="上家出牌 (留空表示'不出')", interactive=False)
            up_btn = gr.Button("上家出牌", interactive=False)
    
    # 下家出牌区域
    with gr.Column() as down_area:
        gr.Markdown("### 下家出牌")
        with gr.Row():
            down_input = gr.Textbox(label="下家出牌 (留空表示'不出')", interactive=False)
            down_btn = gr.Button("下家出牌", interactive=False)
    
    # 事件处理
    submit_btn.click(
        fn=initialize_game,
        inputs=[iniCard, bottomCard, place],
        outputs=[recording, hand_cards, ai_suggestion, 
                player_input, up_input, down_input,
                player_btn, up_btn, down_btn]
    )
    
    player_btn.click(
        fn=process_move,
        inputs=[player_input, gr.State("player")],
        outputs=[recording, hand_cards, ai_suggestion,
                player_input, up_input, down_input,
                player_btn, up_btn, down_btn]
    )
    
    up_btn.click(
        fn=process_move,
        inputs=[up_input, gr.State("up")],
        outputs=[recording, hand_cards, ai_suggestion,
                player_input, up_input, down_input,
                player_btn, up_btn, down_btn]
    )
    
    down_btn.click(
        fn=process_move,
        inputs=[down_input, gr.State("down")],
        outputs=[recording, hand_cards, ai_suggestion,
                player_input, up_input, down_input,
                player_btn, up_btn, down_btn]
    )

demo.launch(inbrowser=True)

Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


C:\Users\Administrator\Desktop\Homework\AI\homework\ddz\DouZero_For_HappyDouDiZhu-master\douzero\evaluation\deep_agent.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  p

In [ ]:
import gradio as gr
from douzero.env.game import GameEnv
from douzero.evaluation.deep_agent import DeepAgent

# 常量定义
EnvCard2RealCard = {3: '3', 4: '4', 5: '5', 6: '6', 7: '7',
                    8: '8', 9: '9', 10: 'T', 11: 'J', 12: 'Q',
                    13: 'K', 14: 'A', 17: '2', 20: 'X', 30: 'D'}

RealCard2EnvCard = {'3': 3, '4': 4, '5': 5, '6': 6, '7': 7,
                    '8': 8, '9': 9, 'T': 10, 'J': 11, 'Q': 12,
                    'K': 13, 'A': 14, '2': 17, 'X': 20, 'D': 30}

AllEnvCard = [3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7,
              8, 8, 8, 8, 9, 9, 9, 9, 10, 10, 10, 10, 11, 11, 11, 11, 12,
              12, 12, 12, 13, 13, 13, 13, 14, 14, 14, 14, 17, 17, 17, 17, 20, 30]

allowed_cards = ["A", "2", "3", "4", "5", "6", "7", "8", "9", "T", "J", "Q", "K", "X", "D"]

custom_css = """
.gradio-container {
    max-width: 900px;
    margin: 0 auto !important;
    padding: 20px !important;
}
#title {
    text-align: center;
    font-size: 24px;
    margin-bottom: 30px !important;
}
#subtitle {
    text-align: left;
    font-size: 16px;
    margin-bottom: 2px !important;
}
.player-area {
    border: 1px solid #ccc;
    border-radius: 5px;
    padding: 10px;
    margin-bottom: 10px;
}
.landlord {
    background-color: #ffebee;
}
.farmer {
    background-color: #e8f5e9;
}
.card-library {
    font-family: monospace;
    margin-top: 10px;
}
"""

class DouDiZhuGame:
    def __init__(self):
        self.card_play_model_path_dict = {
            'landlord': "baselines/douzero_WP/landlord.ckpt",
            'landlord_up': "baselines/douzero_WP/landlord_up.ckpt",
            'landlord_down': "baselines/douzero_WP/landlord_down.ckpt"
        }
        self.env = None
        self.user_position = None
        self.play_order = None
        self.consecutive_passes = 0
        self.first_round = True
        self.last_played_position = None
        self.must_play = False
        self.recording = []
        self.current_hand_cards = ""
        self.current_suggestion = ""
        self.game_over = False
        self.winner = None
        self.card_library = None

    def initialize_game(self, user_hand_cards_real, three_landlord_cards_real, user_position_code):
        """初始化游戏"""
        self.recording = []
        self.user_position = ['landlord_up', 'landlord', 'landlord_down'][user_position_code]
        
        # 初始化牌库
        self.card_library = {'3': 4, '4': 4, '5': 4, '6': 4, '7': 4, '8': 4, '9': 4, 'T': 4,
                           'J': 4, 'Q': 4, 'K': 4, 'A': 4, '2': 4, 'X': 1, 'D': 1}
        
        # 扣除玩家手牌
        for card in user_hand_cards_real:
            self.card_library[card] -= 1
        
        # 如果玩家是地主，扣除底牌
        if self.user_position == "landlord":
            for card in three_landlord_cards_real:
                self.card_library[card] -= 1
        
        # 转换手牌格式
        self.user_hand_cards_env = [RealCard2EnvCard[c] for c in list(user_hand_cards_real)]
        self.three_landlord_cards_env = [RealCard2EnvCard[c] for c in list(three_landlord_cards_real)]
        
        # 如果玩家是地主，将底牌加入手牌
        if self.user_position == "landlord":
            self.user_hand_cards_env.extend(self.three_landlord_cards_env)
            user_hand_cards_real += three_landlord_cards_real
        
        # 计算其他玩家的手牌
        self.other_hand_cards = []
        for i in set(AllEnvCard):
            self.other_hand_cards.extend([i] * (AllEnvCard.count(i) - self.user_hand_cards_env.count(i)))

        # 分配手牌
        if self.user_position == "landlord":
            landlord_up_cards = self.other_hand_cards[:17]
            landlord_down_cards = self.other_hand_cards[17:]
        elif self.user_position == "landlord_up":
            landlord_cards = self.other_hand_cards[:17] + self.three_landlord_cards_env
            landlord_down_cards = self.other_hand_cards[17:34]
        else:  # landlord_down
            landlord_cards = self.other_hand_cards[:17] + self.three_landlord_cards_env
            landlord_up_cards = self.other_hand_cards[17:34]

        self.card_play_data_list = {
            'three_landlord_cards': self.three_landlord_cards_env,
            'landlord_up': landlord_up_cards if self.user_position != "landlord_up" else self.user_hand_cards_env,
            'landlord': landlord_cards if self.user_position != "landlord" else self.user_hand_cards_env,
            'landlord_down': landlord_down_cards if self.user_position != "landlord_down" else self.user_hand_cards_env
        }

        # 设置出牌顺序
        self.play_order = 0 if self.user_position == "landlord" else 1 if self.user_position == "landlord_up" else 2

        # 创建游戏环境
        ai_players = [0, 0]
        ai_players[0] = self.user_position
        ai_players[1] = DeepAgent(self.user_position, self.card_play_model_path_dict[self.user_position])
        
        self.env = GameEnv(ai_players)
        self.env.card_play_init(self.card_play_data_list)
        
        # 重置游戏状态
        self.consecutive_passes = 0
        self.first_round = True
        self.last_played_position = None
        self.must_play = False
        self.game_over = False
        self.winner = None
        
        # 记录初始状态
        position_name = "地主" if self.user_position == "landlord" else "农民(上家)" if self.user_position == "landlord_up" else "农民(下家)"
        self.recording.append(f"游戏开始！你的位置: {position_name}")
        self.update_current_hand_cards()
        return "游戏初始化成功！"

    def update_current_hand_cards(self):
        """更新当前手牌显示"""
        if self.env and hasattr(self.env, 'info_sets'):
            self.current_hand_cards = ''.join([EnvCard2RealCard[c] for c in sorted(
                self.env.info_sets[self.user_position].player_hand_cards)])
        else:
            self.current_hand_cards = ""

    def update_card_library(self, played_cards):
        """更新牌库"""
        if not played_cards:  # 不出牌
            return
            
        for card in played_cards:
            self.card_library[card] -= 1

    def get_card_library_display(self):
        """获取牌库显示文本"""
        remaining_cards = []
        for card, count in sorted(self.card_library.items()):
            if count > 0:
                remaining_cards.append(f"{card}:{count}")
        return " ".join(remaining_cards)

    def get_ai_suggestion(self):
        """获取AI建议"""
        if not self.env or self.play_order != 0:
            return ""
            
        action_message = self.env.step(self.user_position)
        action_str = action_message["action"] if action_message["action"] else "不出"
        win_rate = action_message['win_rate']
        
        self.current_suggestion = f"{action_str} (胜率: {win_rate})"
        return self.current_suggestion

    def play_move(self, move_input, player_type):
        """处理出牌"""
        if self.game_over:
            return "游戏已结束，请重新开始新游戏"
            
        # 确定当前应该出牌的玩家
        current_player = ""
        if player_type == "player":
            current_player = self.user_position
        elif player_type == "up":
            current_player = "landlord_up" if self.user_position != "landlord_up" else None
        elif player_type == "down":
            current_player = "landlord_down" if self.user_position != "landlord_down" else None
            
        if not move_input:  # 不出
            if self.must_play:
                return "错误: 必须出牌，不能选择'不出'！"
            
            self.consecutive_passes += 1
            player_name = "地主" if current_player == "landlord" else "农民(上家)" if current_player == "landlord_up" else "农民(下家)"
            self.recording.append(f"[{player_name}出牌]: 不出")
            
            # 如果两个玩家都pass了，回到最后出牌的玩家
            if self.consecutive_passes >= 2 and self.last_played_position is not None:
                self.play_order = {'landlord':0, 'landlord_up':1, 'landlord_down':2}[self.last_played_position]
                self.consecutive_passes = 0
                self.must_play = True
                last_player_name = "地主" if self.last_played_position == "landlord" else "农民(上家)" if self.last_played_position == "landlord_up" else "农民(下家)"
                self.recording.append(f"[新回合] 由{last_player_name}开始新一轮出牌 (必须出牌)")
                return "\n".join(self.recording)
        else:
            # 验证牌型
            if not self.validate_card_pattern(move_input):
                return "错误: 牌型不符合规则！"
                
            # 转换牌型
            played_cards_env = [RealCard2EnvCard[c] for c in list(move_input)]
            self.env.step(current_player, played_cards_env)
            
            # 更新牌库
            self.update_card_library(move_input)
            
            self.consecutive_passes = 0
            self.last_played_position = current_player
            self.must_play = False
            player_name = "地主" if current_player == "landlord" else "农民(上家)" if current_player == "landlord_up" else "农民(下家)"
            self.recording.append(f"[{player_name}出牌]: {move_input}")
        
        self.first_round = False
        
        # 更新出牌顺序 - 根据斗地主规则调整
        if self.user_position == "landlord":
            # 地主 -> 下家(landlord_down) -> 上家(landlord_up)
            self.play_order = 2 if self.play_order == 0 else 1 if self.play_order == 2 else 0
        elif self.user_position == "landlord_up":
            # 上家(landlord_up) -> 地主 -> 下家(landlord_down)
            self.play_order = 1 if self.play_order == 0 else 2 if self.play_order == 1 else 0
        else:  # landlord_down
            # 下家(landlord_down) -> 上家(landlord_up) -> 地主
            self.play_order = 0 if self.play_order == 2 else 1 if self.play_order == 0 else 2
        
        self.update_current_hand_cards()
        
        # 检查游戏是否结束
        if self.env.game_over:
            self.game_over = True
            self.winner = '农民' if self.env.winner == 'farmer' else '地主'
            self.recording.append(f"{self.winner}胜，本局结束!")
        
        return "\n".join(self.recording)

    def validate_card_pattern(self, cards_str):
        """验证牌型是否符合规则"""
        if not cards_str:
            return True

        cards = list(cards_str)
        card_count = {}
        for card in cards:
            card_count[card] = card_count.get(card, 0) + 1
        if len(cards) == 2 and set(cards) == {'X', 'D'}:
            return True
        if len(cards) == 1:
            return True
        if len(cards) == 2 and len(card_count) == 1:
            return True
        if len(cards) == 3 and len(card_count) == 1:
            return True
        if len(cards) == 4 and (list(card_count.values()).count(3) == 1 and list(card_count.values()).count(1) == 1):
            return True
        if len(cards) == 4 and len(card_count) == 1:
            return True
        if len(cards) >= 5 and len(card_count) == len(cards) and self.is_sequence(cards):
            return True
        if len(cards) >= 6 and len(cards) % 2 == 0 and all(
                count == 2 for count in card_count.values()) and self.is_sequence(sorted(card_count.keys())):
            return True

        return False

    def is_sequence(self, cards):
        """检查是否是连续的牌"""
        order = ['3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A', '2']
        try:
            card_indices = [order.index(c) for c in cards]
        except ValueError:
            return False
        return all(card_indices[i] + 1 == card_indices[i + 1] for i in range(len(card_indices) - 1))

# 创建游戏实例
game = DouDiZhuGame()

def initialize_game(iniCard, bottomCard, place):
    """初始化游戏处理函数"""
    # 验证输入
    if len(iniCard) != 17:
        raise gr.Error("错误: 初始手牌必须是17张牌！")
    if len(bottomCard) != 3:
        raise gr.Error("错误: 底牌必须是3张牌！")
    for card in list(iniCard):
        if card not in allowed_cards:
            raise gr.Error(f"错误: 无效的牌 '{card}'")
    for card in list(bottomCard):
        if card not in allowed_cards:
            raise gr.Error(f"错误: 无效的牌 '{card}'")
    
    # 转换位置选项
    place_code = 0 if place == "Upper Neighbour (地主上家)" else 1 if place == "Landlord (地主)" else 2
    
    # 初始化游戏
    result = game.initialize_game(iniCard.upper(), bottomCard.upper(), place_code)
    
    # 获取初始手牌和AI建议
    hand_cards = game.current_hand_cards
    ai_suggestion = game.get_ai_suggestion()
    card_library = game.get_card_library_display()
    
    # 根据玩家位置确定哪些输入框应该激活
    player_active = game.play_order == 0
    up_active = game.play_order == 1
    down_active = game.play_order == 2
    
    return (
        result, 
        hand_cards, 
        ai_suggestion,
        card_library,
        gr.Textbox(interactive=player_active),
        gr.Textbox(interactive=up_active),
        gr.Textbox(interactive=down_active),
        gr.Button(interactive=player_active, variant="primary" if player_active else "secondary"),
        gr.Button(interactive=up_active, variant="primary" if up_active else "secondary"),
        gr.Button(interactive=down_active, variant="primary" if down_active else "secondary")
    )

def process_move(move_input, player_type):
    """处理出牌函数"""
    if not game.env:
        return "错误: 游戏未初始化，请先提交初始手牌", "", "", "", "", "", "", "", "", ""
    
    move_input = move_input.upper() if move_input else ""
    for card in list(move_input):
        if card not in allowed_cards and move_input:
            return "错误: 牌型包含无效的牌", "", "", "", "", "", "", "", "", ""
    
    # 处理出牌
    recording = game.play_move(move_input, player_type)
    hand_cards = game.current_hand_cards
    
    # 如果是玩家回合，获取AI建议
    ai_suggestion = ""
    if game.play_order == 0 and not game.game_over:
        ai_suggestion = game.get_ai_suggestion()
    
    # 获取牌库状态
    card_library = game.get_card_library_display()
    
    # 根据当前出牌顺序确定哪些输入框应该激活
    player_active = game.play_order == 0 and not game.game_over
    up_active = game.play_order == 1 and not game.game_over
    down_active = game.play_order == 2 and not game.game_over
    
    return (
        recording, 
        hand_cards, 
        ai_suggestion,
        card_library,
        gr.Textbox(interactive=player_active, value=""),
        gr.Textbox(interactive=up_active, value=""),
        gr.Textbox(interactive=down_active, value=""),
        gr.Button(interactive=player_active, variant="primary" if player_active else "secondary"),
        gr.Button(interactive=up_active, variant="primary" if up_active else "secondary"),
        gr.Button(interactive=down_active, variant="primary" if down_active else "secondary")
    )

with gr.Blocks(css=custom_css) as demo: 
    # 标题
    gr.Markdown("""<h1 id="title">斗地主AI辅助工具</h1>""")
    
    # 初始化部分
    gr.Markdown("""<h3 id="subtitle">初始设置</h3>""")
    with gr.Row():
        iniCard = gr.Textbox(label="你的手牌 (17张)", max_length=17)
        bottomCard = gr.Textbox(label="底牌 (3张)", max_length=3)
    place = gr.Radio(
        choices=["Upper Neighbour (地主上家)", "Landlord (地主)", "Lower Neighbour (地主下家)"], 
        label="你的位置", 
        value="Landlord (地主)"
    )
    submit_btn = gr.Button("开始游戏", variant="primary")
    
    # 游戏记录部分
    gr.Markdown("""<h3 id="subtitle">游戏记录</h3>""")
    recording = gr.Textbox(label="游戏记录", interactive=False, lines=10)
    
    # 游戏状态部分
    gr.Markdown("""<h3 id="subtitle">游戏状态</h3>""")
    with gr.Row():
        hand_cards = gr.Textbox(label="当前手牌", interactive=False)
        ai_suggestion = gr.Textbox(label="AI建议", interactive=False)
    card_library = gr.Textbox(label="剩余牌库", interactive=False, elem_classes="card-library")
    
    # 玩家出牌区域
    with gr.Column() as player_area:
        gr.Markdown("### 玩家出牌")
        with gr.Row():
            player_input = gr.Textbox(label="你的出牌 (留空表示'不出')", interactive=False)
            player_btn = gr.Button("出牌", interactive=False)
    
    # 上家出牌区域
    with gr.Column() as up_area:
        gr.Markdown("### 上家出牌")
        with gr.Row():
            up_input = gr.Textbox(label="上家出牌 (留空表示'不出')", interactive=False)
            up_btn = gr.Button("上家出牌", interactive=False)
    
    # 下家出牌区域
    with gr.Column() as down_area:
        gr.Markdown("### 下家出牌")
        with gr.Row():
            down_input = gr.Textbox(label="下家出牌 (留空表示'不出')", interactive=False)
            down_btn = gr.Button("下家出牌", interactive=False)
    
    # 事件处理
    submit_btn.click(
        fn=initialize_game,
        inputs=[iniCard, bottomCard, place],
        outputs=[recording, hand_cards, ai_suggestion, card_library,
                player_input, up_input, down_input,
                player_btn, up_btn, down_btn]
    )
    
    player_btn.click(
        fn=process_move,
        inputs=[player_input, gr.State("player")],
        outputs=[recording, hand_cards, ai_suggestion, card_library,
                player_input, up_input, down_input,
                player_btn, up_btn, down_btn]
    )
    
    up_btn.click(
        fn=process_move,
        inputs=[up_input, gr.State("up")],
        outputs=[recording, hand_cards, ai_suggestion, card_library,
                player_input, up_input, down_input,
                player_btn, up_btn, down_btn]
    )
    
    down_btn.click(
        fn=process_move,
        inputs=[down_input, gr.State("down")],
        outputs=[recording, hand_cards, ai_suggestion, card_library,
                player_input, up_input, down_input,
                player_btn, up_btn, down_btn]
    )

demo.launch(inbrowser=True)